<a href="https://colab.research.google.com/github/AniketJoshi111/STC-repo/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
from google.colab import files
uploaded = files.upload()


Saving preprocessed_stc.csv to preprocessed_stc.csv


In [3]:
data = pd.read_csv('preprocessed_stc.csv')
print(data.head())


  Previous_Grades  Attendance Class_Participation  Study_Hours  \
0               A          81              Medium          8.0   
1               C          93              Medium         12.0   
2               A          86                 Low         15.0   
3               A          84                 Low         13.0   
4               C          73                High         10.0   

  Financial_Status School_Environment Professor_Quality  Screen_Time Tutoring  \
0              Low           Negative            Medium          3.0      Yes   
1              Low           Positive            Medium          7.0       No   
2           Medium           Positive              High          9.0      Yes   
3             High           Positive              High          5.0       No   
4              Low           Positive              High          2.0       No   

  Lack_of_Interest  Time_Wasted_on_Social_Media Grades  
0              Low                            2      B  
1 

In [4]:
print(data.isnull().sum())


Previous_Grades                1
Attendance                     0
Class_Participation            0
Study_Hours                    1
Financial_Status               0
School_Environment             0
Professor_Quality              0
Screen_Time                    2
Tutoring                       2
Lack_of_Interest               0
Time_Wasted_on_Social_Media    0
Grades                         3
dtype: int64


In [5]:
data = data.dropna(subset=['Previous_Grades', 'Study_Hours', 'Screen_Time', 'Tutoring', 'Grades'])


In [6]:
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Grades')
#removed grades becoz it will be used for testing


In [7]:
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Grades')


In [8]:
le = LabelEncoder()
for col in categorical_cols:
    data[col] = le.fit_transform(data[col])

In [9]:
X = data.drop('Grades', axis=1)
y = data['Grades']


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [27]:
best_rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
svm_model = SVC(kernel='rbf',probability=True)

# Train a third model (e.g., Decision Tree)
dt_model = DecisionTreeClassifier(criterion='entropy',max_depth=10)

nb_model = GaussianNB(var_smoothing=1.0)
knn_model = KNeighborsClassifier(weights='uniform',n_neighbors=11,metric="euclidean",algorithm='auto')

#meta model
meta_model = LogisticRegression()

#meta model : Logisitic regression

enc = OneHotEncoder(handle_unknown='ignore')


In [29]:


# Initialize StratifiedKFold with 10 splits
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


# Arrays to store fold accuracies
rf_accuracies = []
svm_accuracies = []
dt_accuracies = []
nb_accuracies = []
knn_accuracies = []
stack_accuracies = []

# Loop through each fold
for train_index, test_index in skf.split(X_train, y_train):
    # Split the data into training and testing sets for this fold
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Train each model on the training fold
    best_rf_model.fit(X_train_fold, y_train_fold)
    svm_model.fit(X_train_fold, y_train_fold)
    dt_model.fit(X_train_fold, y_train_fold)
    nb_model.fit(X_train_fold, y_train_fold)
    knn_model.fit(X_train_fold, y_train_fold)


    # Make predictions on the test fold
    rf_pred = best_rf_model.predict(X_test_fold)
    svm_pred = svm_model.predict(X_test_fold)
    dt_pred = dt_model.predict(X_test_fold)
    nb_pred = nb_model.predict(X_test_fold)
    knn_pred = knn_model.predict(X_test_fold)


    # Make predictions on the training fold for the meta-model
    rf_pred_train = best_rf_model.predict(X_train_fold)
    svm_pred_train = svm_model.predict(X_train_fold)
    dt_pred_train = dt_model.predict(X_train_fold)
    nb_pred_train = nb_model.predict(X_train_fold)
    knn_pred_train = knn_model.predict(X_train_fold)

    # Stack the predictions
    stacked_features_train = np.column_stack((rf_pred_train, svm_pred_train, dt_pred_train,nb_pred_train,knn_pred_train))

    # Encode the stacked features

    stacked_features_encoded_train = enc.fit_transform(stacked_features_train).toarray()


    stacked_features = np.column_stack((rf_pred, svm_pred, dt_pred,nb_pred,knn_pred))

    # Encode the stacked features for the test fold
    stacked_features_encoded = enc.transform(stacked_features).toarray()


    # meta_model.fit(stacked_features_encoded, y_train_fold)

    # meta_pred = meta_model.predict(stacked_features_encoded)

    # Calculate stacking accuracy and store it

    rf_accuracy = accuracy_score(y_test_fold, rf_pred)
    svm_accuracy = accuracy_score(y_test_fold, svm_pred)
    dt_accuracy = accuracy_score(y_test_fold, dt_pred)
    nb_accuracy = accuracy_score(y_test_fold, nb_pred)
    knn_accuracy = accuracy_score(y_test_fold, knn_pred)
    #stack_accuracy = accuracy_score(y_test_fold, meta_pred)


    rf_accuracies.append(rf_accuracy)
    svm_accuracies.append(svm_accuracy)
    dt_accuracies.append(dt_accuracy)
    nb_accuracies.append(nb_accuracy)
    knn_accuracies.append(knn_accuracy)
    #stack_accuracies.append(stack_accuracy)

print("Random Forest Avg Accuracy:", np.mean(rf_accuracies))
print("SVM Avg Accuracy:", np.mean(svm_accuracies))
print("Decision Tree Avg Accuracy:", np.mean(dt_accuracies))
print("Naive Bayes Avg Accuracy:", np.mean(nb_accuracies))
print("KNN Avg Accuracy:", np.mean(knn_accuracies))
#print("Stacking accuracy:",np.mean(stack_accuracies))
# (If using a meta-model, print its average accuracy here)


Random Forest Avg Accuracy: 0.32368856744510277
SVM Avg Accuracy: 0.32101777622865113
Decision Tree Avg Accuracy: 0.3210700592540955
Naive Bayes Avg Accuracy: 0.29933339142558385
KNN Avg Accuracy: 0.3388506448239805


In [33]:
rf_pred_test = best_rf_model.predict(X_test)
svm_pred_test = svm_model.predict(X_test)
dt_pred_test = dt_model.predict(X_test)
nb_pred_test = nb_model.predict(X_test)
knn_pred_test = knn_model.predict(X_test)

stacked_test_features = np.column_stack((rf_pred_test, svm_pred_test,dt_pred_train,nb_pred_test,knn_pred_test))

stacked_test_features_encoded = enc.transform(stacked_test_features).toarray()
final_predictions = meta_model.predict(stacked_test_features)
print("Final Accuracy:", accuracy_score(y_test, final_predictions))

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1517 and the array at index 2 has size 1366

In [ ]:
importances = model.feature_importances_
feature_importance = pd.Series(importances, index=X.columns)
feature_importance.nlargest(10).plot(kind='barh')



In [ ]:
# Remove the mean_squared_error and r2_score calculation as they are not appropriate for classification problems
# Predictions
y_pred = model.predict(X_test)


# Print the results you have already calculated in the previous code block
print("Accuracy:", accuracy_score(y_test, y_pred))
